In [24]:
#import liabraries
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [25]:
#Data downloded from kaggel
true = pd.read_csv('/Users/namratabhoyar/Downloads/true.csv.zip')
fake = pd.read_csv('/Users/namratabhoyar/Downloads/fake.csv.zip')

In [26]:
true.sample(5)

,title,text,subject,date
1016,Top Republican senator testifies for Menendez ...,(Reuters) - Republican U.S. Senator Lindsay Gr...,politicsNews,"October 26, 2017"
19655,France's Macron seeks to play mediation role b...,UNITED NATIONS (Reuters) - French President Em...,worldnews,"September 19, 2017"
7687,Factbox: Wall Street's take on possible impact...,(Reuters) - U.S. Democratic presidential candi...,politicsNews,"October 26, 2016"
19109,Qatar Says Trump wants to fix Gulf crisis with...,PARIS (Reuters) - Qatar s foreign minister sai...,worldnews,"September 25, 2017"
16708,"After U.N. troubles, Haiti wary of new justice...","PORT-AU-PRINCE, Haiti (Reuters) - A new United...",worldnews,"October 23, 2017"


In [27]:
fake.sample(5)

,title,text,subject,date
13040,TAKE THEM OFF OR PAY HUGE FINE! #NFL Won’t All...,We re still waiting for the NFL to make a stat...,politics,"Sep 11, 2016"
4183,WATCH: Tom Hanks Uses This Perfect Analogy To...,Tom Hanks is definitely voting for Hillary Cli...,News,"October 15, 2016"
6930,Major Corporation Leaves South Carolina After...,South Carolina Republicans just introduced an ...,News,"April 13, 2016"
17374,BREAKING: BALTIMORE GOT BUCKET LOADS OF MONEY ...,Holy smokes! That s billion with a B! Where di...,Government News,"May 4, 2015"
23041,U.S. Inauguration: Historic Day Marks Beginnin...,"Mark Anderson American Free PressWASHINGTON,...",Middle-east,"January 21, 2017"


In [28]:
#get row's and column's for true
true.shape

(21417, 4)

In [29]:
#get row's and column's for fake
fake.shape

(23481, 4)

In [30]:
true['target'] = 0
fake['target'] = 1

In [31]:
#Manual Testing for Both the Dataset
data_fake_manual_testing = fake.tail(10)
for i in range (23480, 21470, -1):
    fake.drop([i], axis = 0, inplace = True)
data_true_manual_testing = true.tail(10)
for i in range (21416, 21406, -1):
    true.drop([i], axis = 0, inplace = True)


In [32]:
data = pd.concat([true, fake], axis=0)
data = data.drop(['date', 'title'], axis=1)

data = data.sample(frac=1)

data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

In [33]:
print(data.columns)

Index(['text', 'subject', 'target'], dtype='object')


In [34]:
print(data.head())

                                                text       subject  target
0  The outspoken liberal comedian posed for a pho...      politics       1
1  WASHINGTON (Reuters) - There will be no planne...  politicsNews       0
2  On Sunday, Donald Trump proved that he doesn t...          News       1
3  Wow! Talk about clueless! Austen Fletcher appr...      politics       1
4  Clinton News Network  a propaganda arm of the ...      politics       1


In [35]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '' , text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [36]:
#assigning X and Y
data['text'] = data['text'].apply(wordopt)

x = data['text'].values
y = data['target'].values

In [37]:
#Train Test Split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)



In [38]:
vectorization = TfidfVectorizer()
vectoriztion = vectorization  
xv_train = vectoriztion.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [39]:
#import first model
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [40]:
# Model Accuracy and Classification Report
pred_lr = LR.predict(xv_test)
LR.score(xv_test, y_test)
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      5364
           1       0.99      0.98      0.99      5356

    accuracy                           0.99     10720
   macro avg       0.99      0.99      0.99     10720
weighted avg       0.99      0.99      0.99     10720



In [41]:
#creating second model
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [42]:
#Checking the Model Accuracy and Classification Report
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5364
           1       0.99      1.00      1.00      5356

    accuracy                           1.00     10720
   macro avg       1.00      1.00      1.00     10720
weighted avg       1.00      1.00      1.00     10720



In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} ".format(
        output_lable(pred_LR[0]),
        output_lable(pred_DT[0])
    ))


In [ ]:
#news = str(input())
#manual_testing(news)